In [9]:
import numpy as np

In [10]:
def tsm(image):

    threshold = np.mean(image)

    epsilon = 1e-6

    m1 = 0
    m2 = 0

    while True:

        g1 = image[image > threshold]
        g2 = image[image <= threshold]

        m1 = np.mean(g1) if len(g1) > 0 else m1
        m2 = np.mean(g2) if len(g2) > 0 else m2

        new_threshold = (m1 + m2) / 2

        if abs(new_threshold - threshold) < epsilon:
            break

        threshold = new_threshold

    return threshold


In [11]:
def otsu_threshold(image):

    hist = np.histogram(image, bins=256, range=(0, 255))[0]

    hist = hist / hist.sum()

    csum = np.cumsum(hist)

    csum_sq = np.cumsum(hist * hist)

    csum_xy = np.cumsum(hist * np.arange(256))

    mean = csum_xy[-1] / csum[-1]

    max_var_bt = 0
    threshold = 0

    for i in range(1, 256):

        p0, p1 = csum[i], 1 - csum[i]
        m0 = csum_xy[i]/p0 if p0!=0 else np.finfo(float).eps 
        m1 = (csum_xy[-1] - csum_xy[i])/p1 if p1!=0 else np.finfo(float).eps


        var_bt = p0 * (m0 - mean) ** 2 + p1 * (m1 - mean) ** 2

        if var_bt > max_var_bt:
            max_var_bt = var_bt
            threshold = i

    return threshold


In [12]:
def read_pgm(image_file):
    with open(image_file, 'r') as file:
        image = file.read().splitlines()
    for pixel in range(3,len(image)):
        image[pixel] = int(image[pixel])
    image_matrix = np.array(image[4:]).reshape(int(image[2].split()[1]),int(image[2].split()[0]))
    return (image_matrix, image[:4])

In [13]:
def matrix2pgm(image_matrix,initial,output_image_file_name = str(np.random.randint(1,100000000000)) + '_default.pgm'):    
    with open(output_image_file_name, 'w') as file:
        file.writelines("% s\n" % pixel for pixel in initial)
        file.writelines("% s\n" % int(abs(pixel)) for pixel in image_matrix.flatten())

In [14]:
image_file = 'z.pgm'
image = read_pgm(image_file)

In [15]:
threshold = otsu_threshold(image[0])
threshold

191

In [16]:
thresholded_image = np.where(image[0] > threshold, 255, 0)

In [17]:
matrix2pgm(thresholded_image,image[1],output_image_file_name = 'otsu_'+image_file)

In [18]:
threshold1 = tsm(image[0])
threshold1

180.4561668320934

In [19]:
thresholded_image1 = np.where(image[0] > threshold1, 255, 0)

In [20]:
matrix2pgm(thresholded_image1,image[1],output_image_file_name = 'tsm_'+image_file)